<a href="https://colab.research.google.com/github/nkkchem/SCA24_Hackathon/blob/main/Agent_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Large Language Model Agents in LangChain





This is a brief introduction for how to use `LangChain` for the building of Large Language Model agents.

LLM-Agents are essentially LLMs that are given tools specific to their desired task. This means that if you want an LLM to be able to configure a database, or access the internet, Agents are how you go about this task.

This tutorial will step you through a potential application of LLMs - a pandas dataframe manager. We will see how to load in a local LLM through `vllm` as well as use an API-based model such as gpt-4, gemini-pro, or claude.

This tutorial is adapted from a tutorial build by PineCone-IO at https://github.com/pinecone-io/examples

---

To start, let's install the packages we will be using for this example.

In [33]:
!pip install -qU torch==2.3.0 vllm==0.4.2 langchain langchain_community langchain_core langchain_experimental openai google-search-results pandas langchain_openai langchain_google_genai

## Pandas Agent - Setup

Pretend we have an pandas dataframe (or multiple) that we would like to set up an LLM to interact with. This type of integration can provide a natural language way of gaining insights from your data. This can assist non-experts in utilizing your groups data to form insights.

We will first need to create an pandas dataframe, then set up an LLM to interface with the data.

⚠️ In this tutorial we will try and use local Google Colab GPU resources, but the free tier of colab prohibits larger models and may decrease the reliability of the LLM-Agents. For this situation, we will provide an additional loader for the OpenAI API models, but you will need an API-key to utilize this resoruce. ⚠️

----

## The Large Language Models

In the following cells, we will create an LLM while pulling an example pandas dataframe from github. We will first provide instructions for spinning up a `vllm`-based LLM which pulls in models from `HuggingFace`. Then we will show the equivalent steps for OpenAI and Google.

IMPORTANT: The rest of this notebook should only use one of the two approaches. Choose one of the 3 below cells to run.

In [ ]:
# RUN THIS CELL FOR VLLM MODELS

import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_community.llms import VLLM

llm = VLLM(
    model="microsoft/Phi-3-mini-4k-instruct", # Set some HF model name, we prefer smaller models since they have to fit in 15 GB of vram
    trust_remote_code=True,  # mandatory for hf models
    temperature=0,
    dtype="float16", # This is only required when using the T4 GPU on Colab
)

df = pd.read_csv(
    "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
)

In [2]:
# RUN THIS CELL FOR OPENAI MODELS

import os
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) # You need to have you OPENAI_API Key set. If not add OPENAI_API_KEY to the secrets tab on the left side of the screen (click the key)

df = pd.read_csv(
    "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
)

In [34]:
# RUN THIS CELL FOR GOOGLE MODELS

from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)

df = pd.read_csv(
    "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
)

----

## The Agent

Now that we have our `llm` loaded, we can add that chain to an `AgentExecutor` type which will provide the background thought process that the agent should be able to handle.

There are a few different pre-made Agent types that LangChain provides:

1. Zero Shot React
2. Conversational React
3. React Docstore

or variations of these, which can be found in the documentation [here](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html)


---

We are going to use the Zero Shot React agent type. This type of agent is used to perform 'zero shot' tasks on the input. That means that we will not have several, interdependent interactions but only one. In other words, this agent will have no memory.

In [35]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType

agent_executor = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:249: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


### Single DataFrame

Now we can run some example questions to see how well the agent handles the example dataframe.

In [36]:
agent_executor.invoke(
    "how many rows are there?"
)



> Entering new AgentExecutor chain...
Thought: I can use the `shape` attribute to get the number of rows and columns
Action: python_repl_ast
Action Input: df.shape(891, 12)Final Answer: 891

> Finished chain.


{'input': 'how many rows are there?', 'output': '891'}

In [37]:
agent_executor.run("how many people have more than 3 siblings")



> Entering new AgentExecutor chain...
Thought: I should filter the dataframe to only include people with more than 3 siblings, then count the number of rows in the resulting dataframe
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].count()PassengerId    30
Survived       30
Pclass         30
Name           30
Sex            30
Age            23
SibSp          30
Parch          30
Ticket         30
Fare           30
Cabin           0
Embarked       30
dtype: int64Final Answer: 30

> Finished chain.


'30'

In [38]:
agent_executor.invoke("whats the square root of the average age?")



> Entering new AgentExecutor chain...
Thought: I should first find the average age, then find the square root of that value
Action: python_repl_ast
Action Input: df['Age'].mean()29.69911764705882Action: python_repl_ast
Action Input: (df['Age'].mean())**0.55.449689683556195Final Answer: 5.449689683556195

> Finished chain.


{'input': 'whats the square root of the average age?',
 'output': '5.449689683556195'}

### Multiple DataFrames

Now that we show that we can run the LLM Agent over a single dataframe, we can add in a slightly altered version of the dataframe and show how we can pass multiple dataframes to the agent.

In [39]:
df1 = df.copy()
df1["Age"] = df1["Age"].fillna(df1["Age"].mean())

In [40]:
agent_executor = create_pandas_dataframe_agent(
    llm=llm,
    df=[df,df1],
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:249: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [47]:
agent_executor.invoke("Does one dataframe have more information in the age column?")



> Entering new AgentExecutor chain...
Thought: I should check the number of non-null values in the age column of each dataframe
Action: python_repl_ast
Action Input: df1.age.notnull().sum()AttributeError: 'DataFrame' object has no attribute 'age'I should check the number of non-null values in the age column of each dataframe
Action: python_repl_ast
Action Input: df1['Age'].notnull().sum()714I should check the number of non-null values in the age column of each dataframe
Action: python_repl_ast
Action Input: df2['Age'].notnull().sum()891I now know the final answer
Final Answer: df2 has more information in the age column

> Finished chain.


{'input': 'Does one dataframe have more information in the age column?',
 'output': 'df2 has more information in the age column'}

## Scientific Example - Dataframe of Cells

Let's look at a scientific example. We will load in a dataframe that contains information on cells. These cells are located in various tissue types and organisms. We can use this to illustrate how LLM-agents might be integrated into scientific databases.


In [48]:
df = pd.read_parquet("hf://datasets/pankajrajdeo/PubChem_cell/PubChem_cell_text.parquet")

In [49]:
agent_executor = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:249: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


In [50]:
agent_executor.invoke("Count how many tissue types there are.")



> Entering new AgentExecutor chain...
Thought: I should use the `nunique` function to count the number of unique values in the `tissue` column.
Action: python_repl_ast
Action Input: df['tissue'].nunique()57Final Answer: 57

> Finished chain.


{'input': 'Count how many tissue types there are.', 'output': '57'}

In [51]:
agent_executor.invoke("What is the most common organism?")



> Entering new AgentExecutor chain...
Thought: I should use the `value_counts()` function to count the number of occurrences of each organism.
Action: python_repl_ast
Action Input: df['organism'].value_counts()organism
Homo sapiens (human)                                 2403
Cricetulus griseus (Chinese hamster)                  214
Mus musculus (house mouse)                            207
Rattus norvegicus (Norway rat)                         72
Bos taurus (cattle)                                    48
                                                     ... 
Pelodiscus sinensis (Chinese soft-shelled turtle)       1
Gallus gallus (chicken)                                 1
Leontocebus nigricollis (black-and-red tamarin)         1
Tadarida brasiliensis (Brazilian free-tailed bat)       1
Tursiops truncatus (common bottlenose dolphin)          1
Name: count, Length: 89, dtype: int64The most common organism is Homo sapiens (human).
Final Answer: Homo sapiens (human)

> Finished chain.


{'input': 'What is the most common organism?',
 'output': 'Homo sapiens (human)'}